## Import Libraries

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import PIL
import random
from PIL import Image
import json

!pip install segmentation-models-pytorch --quiet
!pip install -U albumentations --quiet

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import torch.nn.functional as F

import torch
import numpy as np
import segmentation_models_pytorch as smp
from torch import nn

import albumentations as albu

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

## DataPath

In [4]:
# DATA_DIR_train = "/content/drive/MyDrive/Colab Notebooks/AI_final_proj/Train_Images/"  #training
# DATA_DIR_mask = "/content/drive/MyDrive/Colab Notebooks/AI_final_proj/Train_Masks/" #training

# DATA_DIR_train = "AI_final_proj/Train_Images/"  #training
# DATA_DIR_mask = "AI_final_proj/Train_Masks/" #training

DATA_DIR_train = "../input/ai-final-dataset/AI_final_proj/Train_Images"  
DATA_DIR_mask = "../input/ai-final-dataset/AI_final_proj/Train_Masks" 

# DATA_DIR_vail = '/data/home/VGH_Seg_IMG_Label/Valid'
# DATA_DIR_test = '/data/home/VGH_Seg_IMG_Label/Test'

# SavePath = "AI_final_proj/model_exp_final/"
SavePath = "./"

path = SavePath
checkpoint = 'unet_effi_b1_50_normaug.pth'

## Hyperparameters & Configs

In [5]:
RESIZE_SIZE = 32*20
BATCH_SIZE = 8
NUM_WORKERS = 2

ENCODER = 'efficientnet-b1'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['stas']
ACTIVATION = 'sigmoid' 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 50

SEED = 1187

DEVICE

## Seed Everything

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(SEED)

## Data Preprocessing

### Get data & formatting

In [7]:
ids = os.listdir(DATA_DIR_train)
# print(ids)
# print(len(ids))

In [8]:
images_list = [os.path.join(DATA_DIR_train, image_id) for image_id in ids]
# display(images_list)
print(len(images_list))
masks_list = [os.path.join(DATA_DIR_mask, mask_id) for mask_id in ids]
# display(masks_list)
print(len(masks_list))

In [9]:
for i in range(len(masks_list)):
  masks_list[i] = masks_list[i].replace('.jpg','.png')
# display(masks_list)

### Splitting Data

In [10]:

x_train, x_valid, y_train, y_valid = train_test_split(images_list, masks_list, test_size=0.2, random_state=SEED)
x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, test_size=0.5, random_state=SEED)


In [11]:
print(len(x_train))
print(len(y_train))
print(len(x_valid))
print(len(y_valid))
print(len(x_test))
print(len(y_test))

In [12]:
# y_train
# # cv2.imread(x_train[0])
# cv2.imread(y_train[0], 0)

### Data Visualization Func

In [13]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 16))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

In [14]:
# from torch.utils.data import DataLoader
# from torch.utils.data import Dataset as BaseDataset
# import torch.nn.functional as F

## Custom Dataset

In [15]:

class Dataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = ['bg', 'stas']
    
    def __init__(
            self, 
            images_list, 
            masks_list, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        # self.ids = os.listdir(images_dir)
        # self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        # self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        self.images_fps = images_list
        self.masks_fps = masks_list
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        # self.masks_fps[i] = self.masks_fps[i].replace('.jpg','.png')
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image.shape != (RESIZE_SIZE, RESIZE_SIZE, 3):
            image = cv2.resize(image, (RESIZE_SIZE, RESIZE_SIZE), interpolation=cv2.INTER_LANCZOS4)
        
        mask = cv2.imread(self.masks_fps[i], 0)
        if mask.shape != (RESIZE_SIZE, RESIZE_SIZE, 3):
            mask = cv2.resize(mask, (RESIZE_SIZE, RESIZE_SIZE), interpolation=cv2.INTER_LANCZOS4)  
        mask = mask.astype('bool')
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.images_fps)

In [16]:
# Lets look at data we have

dataset = Dataset(images_list=images_list, 
                  masks_list=masks_list, 
                  classes=['stas'])

image, mask = dataset[4] # get some sample
visualize(
    image=image, 
    stas_mask=mask.squeeze(),
)

In [17]:
print(image.shape)
print(mask.shape)

In [18]:
mask.squeeze().shape

## Data Augmentation

In [34]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),
        
        albu.VerticalFlip(p=0.5),
        
        albu.HueSaturationValue(p=0.6),
        albu.Sharpen(p=0.5),
        albu.RandomBrightnessContrast(p=0.4), 
        albu.Crop(x_min=0, y_min=0, x_max=RESIZE_SIZE, y_max=RESIZE_SIZE-40, p=0.5),
        albu.PadIfNeeded(RESIZE_SIZE, RESIZE_SIZE),
        
        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.ToGray(p=0.3),
                albu.RandomGamma(p=1),
            ],
            p=0.5,
        ),   
         albu.OneOf(
            [
                albu.Rotate(limit=30,p=1,border_mode=cv2.BORDER_CONSTANT),
                albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=0.5, border_mode=cv2.BORDER_CONSTANT),
            ],
            p=0.8,
             
        ),
        albu.Normalize(),
        
    ]
    return albu.Compose(train_transform)

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


In [35]:


# def get_training_augmentation():
#     train_transform = [

#         albu.HorizontalFlip(p=0.5),
#         albu.Rotate(limit=40,p=1,border_mode=cv2.BORDER_CONSTANT),
#         albu.VerticalFlip(p=0.5),
#         albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=0.5, border_mode=cv2.BORDER_CONSTANT),
        
#         albu.HueSaturationValue(p=0.6),
#         albu.Sharpen(p=0.5),
#         albu.RandomBrightnessContrast(p=0.4),

#         albu.Crop(x_min=0, y_min=0, x_max=RESIZE_SIZE, y_max=RESIZE_SIZE-40, p=0.5),
#         albu.PadIfNeeded(RESIZE_SIZE, RESIZE_SIZE)

        
#     ]
#     return albu.Compose(train_transform)

# def to_tensor(x, **kwargs):
#     return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

### Visualize augmented imgs

In [36]:
#### Visualize resulted augmented images and masks

augmented_dataset = Dataset(
    images_list=images_list, 
    masks_list=masks_list, 
    augmentation=get_training_augmentation(), 
    classes=['stas'],
)

# same image with different random transforms
for i in range(3):
    image, mask = augmented_dataset[1]
    visualize(image=image, mask=mask.squeeze(-1))

## Create model and train

In [37]:

# create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,

)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

### DataLoader

In [38]:
train_dataset = Dataset(
    x_train, 
    y_train, 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

valid_dataset = Dataset(
    x_valid, 
    y_valid, 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

train_loader = DataLoader(train_dataset, pin_memory=True, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
valid_loader = DataLoader(valid_dataset, pin_memory=True, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [39]:
# Dice/F1 score - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# IoU/Jaccard score - https://en.wikipedia.org/wiki/Jaccard_index

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.Fscore(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001 , betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0001, amsgrad=False),
])

In [40]:
# model

In [41]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)



In [42]:

# os.makedirs(path)

In [43]:
import gc
gc.collect()
torch.cuda.empty_cache()    

In [ ]:
max_score = 0

train_loss_list = []
train_fscore_list = []
valid_loss_list = []
valid_fscore_list = []

for i in range(EPOCHS):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    train_loss_list.append(train_logs['dice_loss'])
    train_fscore_list.append(train_logs['fscore'])
    
    valid_loss_list.append(valid_logs['dice_loss'])
    valid_fscore_list.append(valid_logs['fscore'])
    

    if max_score < valid_logs['fscore']:
        max_score = valid_logs['fscore']
        torch.save(model, path+'/'+checkpoint)
        print('Model saved!')

        
#     if i == 35:
#         optimizer.param_groups[0]['lr'] = 1e-5
#         print('Decrease decoder learning rate to 1e-5!')

## Plot Result

In [ ]:
plt.figure(0)
plt.plot(train_fscore_list)
plt.plot(valid_fscore_list)
plt.title('Train Logs')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='lower right')

plt.figure(1)
plt.plot(train_loss_list)
plt.plot(valid_loss_list)
plt.title('Train Logs')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')


plt.show()

## Load  best model

In [ ]:
# load best saved checkpoint
best_model = torch.load(SavePath +'/'+ checkpoint)

In [ ]:
# optimizer.param_groups[0]['lr']
max_score

## Evaluation

In [ ]:
test_dataset = Dataset(
    x_test, 
    y_test, 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=NUM_WORKERS)

In [ ]:
# evaluate model on test set
test_epoch = smp.utils.train.ValidEpoch(
    model=best_model,
    loss=loss,
    metrics=metrics,
    device=DEVICE,
)

logs = test_epoch.run(test_dataloader)

In [ ]:
# test dataset without transformations for image visualization
test_dataset_vis = Dataset(
    x_test, y_test, 
    classes=CLASSES,
)

In [ ]:
out_path= SavePath + "/" + "out_dir"
# os.makedirs(out_path)

### Compare Predictions with Ground Truth

In [ ]:

for i in range(len(test_dataset_vis)):
    name = os.path.basename(test_dataset_vis.masks_fps[i])
    print(name)
    image_vis = test_dataset_vis[i][0].astype('uint8')
    image, gt_mask = test_dataset[i]
    x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())
    len(pr_mask)
#     print(gt_mask.shape)
    zeros = np.zeros((RESIZE_SIZE, RESIZE_SIZE, 3))
    zeros[...,0] = pr_mask
    zeros = cv2.resize(zeros, (1920, 828))
#     print(i)
    visualize( 
            image=image_vis, 
            ground_truth_mask=gt_mask.transpose(1, 2, 0)[...,0], 
            predicted_mask=pr_mask
        )
#     plt.imshow(zeros)
#     plt.show()
#     print(os.path.join(out_path, name.replace('.jpg','.png')))
#     plt.imsave(os.path.join(out_path, name.split.replace('.jpg','.png'), zeros))
    